In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# Setting the URL for the ccny calendar
url_college_schedule = 'https://www.ccny.cuny.edu/registrar/fall'

In [ ]:
# Function to handle date strings and convert them to date objects
def parse_date(date_string):
    if '2022' not in date_string:
        date_string += ", 2021"
    if '-' not in date_string:  # Single date case
        return [datetime.strptime(date_string, "%B %d, %Y").date()]
    # Used to handle the date ranges
    start_end = date_string.split(' - ')
    start_date = datetime.strptime(start_end[0] + ", 2021", "%B %d, %Y").date()
    end_date = start_date.replace(day=int(start_end[1].split(',')[0]))
    return [str(start_date) + " to " + str(end_date)]
